#This has examples of how to build and xml file in python and call gdal.translate as an external command. 

https://github.com/nasa-gibs/gibs-ml/blob/master/download_data.py





In [11]:
import sys
from datetime import datetime, time, date
import os
import subprocess
sys.path.append('/home/jovyan/overcast/gibs-ml')

### This is from a machine learning repo that can be found at https://github.com/nasa-gibs/gibs-ml It is used to build the XML file used in gdalwarp to provide the input data 

In [12]:
from gibs_layer import GIBSLayer

### Set up data to download 

In [13]:
# select either True Color or Bands367
#layer_name='VIIRS_SNPP_CorrectedReflectance_BandsM11_I2_I1'
type='TrueColor'
#type='Band367'
layer_name='MODIS_Terra_CorrectedReflectance_'+type
#layer_name='MODIS_Terra_CorrectedReflectance_Bands367'
# select date to download 
date="2019-06-11"
# select directory where output data will be stored
output_dir='/home/jovyan/shared/data-overcast/modis/'
#output_dir="./"

In [14]:
# output projection parameters Tthis needs to be tuned for the specfic region. A bit messy. 
region='BEAUFORT'
tile_resolution='250m'
proj_str ="'+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-145 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'" 
te_str = '-800000 -2500000 1200000 -400000'
tr_str = '250 250'

# that's the input projection 
epsg='3413'
# set up a GIBS layer used to build the input XML file 
layer = GIBSLayer.get_gibs_layer(layer_name)
get_date = datetime.strptime(date, "%Y-%m-%d")
datestring = get_date.strftime("%Y-%m-%d")
outfile='modis.'+region+'.'+datestring.replace("-","")+ "."+type+'.tif' # needs to be changed so that the type is reflected in the name
outpath = os.path.join(output_dir, outfile)
print("Output will be written to: ",outpath)

Output will be written to:  /home/jovyan/shared/data-overcast/modis/modis.BEAUFORT.20190611.TrueColor.tif


In [15]:
### Generate the XML file used as input

In [16]:
layer.generate_xml(protocol="tms", epsg=epsg, tile_resolution=tile_resolution, datestring=datestring)
xml=layer.gibs_xml

In [17]:
# The date/time data in the xml string produced needs to be preplaced with the actual date/time
xml = xml.replace("{Time}", datestring)
xml = xml.rstrip()  # remove all trailing whitespace
# write output to a temporary xml file 
with open("temp.xml", "w") as f:
    f.write(xml)

In [18]:
# build the command to pass to gdalwarp
cmd_list = ["gdalwarp","-overwrite","-t_srs",proj_str,"-te",te_str,"-tr",tr_str,"temp.xml", outpath]
cmd= ' '.join(cmd_list) # this makes it a single string. Different versions of subprocess.Popen seem to require eithe rlists or strings

### create a subprocess to run gdal... this could probably be replaced by using the Python gdal bindings, but I looked and it seems very complicated. This is easier though not as clean

In [19]:
process = subprocess.Popen(cmd,shell='True',stdout=subprocess.PIPE,stderr=subprocess.PIPE)
out,err = process.communicate()
print(process.returncode, out,err, )
print("Done Writing File :", outpath)
# remove temporary file.. Return condition should be checked
os.remove("temp.xml")

0 b'Creating output file that is 8000P x 8400L.\nProcessing temp.xml [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n' b''
Done Writing File : /home/jovyan/shared/data-overcast/modis/modis.BEAUFORT.20190611.TrueColor.tif


In [20]:
#Above needs some more error checking